In [1]:
import glob
import re
from pathlib import Path

import pandas as pd
from textblob import TextBlob
from transformers import pipeline

In [ ]:
analyzer = pipeline(
    task='text-classification',
    model="cmarkea/distilcamembert-base-sentiment",
    tokenizer="cmarkea/distilcamembert-base-sentiment"
)

In [4]:
df = pd.read_csv('../../data/filtered_cds_data.csv', sep=',')

In [4]:
# DOC FOR FIRST LINES
df['first-line_sentiment'] = 0

In [6]:
# DOC FOR ENTIRE LETTERS
df['sentiment'] = 0

In [6]:
index = 0
for url in df['URL']:
    if df['first-line_sentiment'][index] == 0:
        result = analyzer(
            df['Anfang des Briefes'][index]
        )
        df['first-line_sentiment'][index] = result[0]['label']
    index += 1

In [28]:
index = 0
for file in glob.glob('../../data/transcriptions/*-cleaned.txt'):
    with open(file, 'r') as f:
        pattern = r'[^a-zA-Z0-9\s]'  # Match any character that is not a letter, digit, or whitespace
        cleaned_text = re.sub(pattern, '', f.read())
        blob = TextBlob(cleaned_text)
        sentiment = blob.sentiment.polarity
        if (
                (df['URL'] == f'https://constance-de-salm.de/archiv/#/document/{Path(file).stem.replace("-cleaned", "")}')
                &
                (df['sentiment'] == 0)
        ).any():

            if sentiment > 0:
                df.loc[df[
                           'URL'] == f'https://constance-de-salm.de/archiv/#/document/{Path(file).stem.replace("-cleaned", "")}',
                       'sentiment'] = 'positive'
            elif sentiment < 0:
                df.loc[df[
                           'URL'] == f'https://constance-de-salm.de/archiv/#/document/{Path(file).stem.replace("-cleaned", "")}',
                       'sentiment'] = 'negative'
            else:
                df.loc[df[
                           'URL'] == f'https://constance-de-salm.de/archiv/#/document/{Path(file).stem.replace("-cleaned", "")}',
                       'sentiment'] = 'neutral'


In [31]:
df.to_csv('../../data/filtered_cds_data.csv')